In [3]:
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

In [4]:
def load_data(data_dir = 'data'):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform
    )
    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform
    )
    
    return trainset, testset

In [5]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2  = nn.Conv2d(6, 16, 5)
        self.f1 = nn.Linear(16 * 5 * 5, l1)
        self.f2 = nn.Linear(l1, l2)
        self.f3 = nn.Linear(l2, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.viwew(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        

In [11]:
config = {
    "l1" : tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "l2" : tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "lr" : tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 6, 8])
}

num_samples=10 
max_num_epochs=10
gpus_per_trial=2

scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2
)

repoter = CLIReporter(
    metric_columns=["loss", "accuracy", "training_iteration"]
)

In [12]:
def train_cifar(config, checkpoint_dir=None, data_dir=None):
    
    
    return None